In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
import re
import csv
import os
from datetime import datetime
import glob
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from xml.etree.ElementTree import fromstring 
import shutil
import warnings
import random
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import aiohttp
import asyncio
import nest_asyncio
# from tqdm import tqdm
from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm as tqdm_async

In [2]:
# Remove old data and database

if os.path.exists("data"):
    shutil.rmtree("data")
os.makedirs("data")

# # Create required directories
# directories = ['./data/fighters/', './data/git/']
# for directory in directories:
#     os.makedirs(directory, exist_ok=True)

# # Define CSV files and their columns
# csv_files = {
#     './data/event_urls_sherdog.csv': ['Event_URL'],
#     './data/fighter_id_sherdog.csv': ['Fighter', 'Fighter_ID'],
#     './data/event_data_sherdog.csv': ['Event Name', 'Event Location', 'Event Date', 
#                                       'Fighter 1', 'Fighter 2', 'Weight Class', 
#                                       'Winning Fighter', 'Winning Method', 
#                                       'Winning Round', 'Winning Time', 'Referee'],
#     './data/fighter_info.csv': ['Fighter', 'Nickname', 'Birth Date', 'Nationality', 
#                                 'Hometown', 'Association', 'Weight Class', 'Height', 
#                                 'REACH', 'STANCE', 'Wins', 'Losses', 'Win_Decision', 
#                                 'Win_KO', 'Win_Sub', 'Loss_Decision', 'Loss_KO', 
#                                 'Loss_Sub', 'Sherdog URL', 'BFO URL']
# }

# # Ensure CSV files exist and have headers
# for file_path, columns in csv_files.items():
#     if not os.path.exists(file_path):
#         pd.DataFrame(columns=columns).to_csv(file_path, index=False)

# # Function to download GitHub data
# def download_github_files():
#     files = ["ufc_event_details.csv", "ufc_fight_results.csv", 
#              "ufc_fight_stats.csv", "ufc_fighter_tott.csv"]
#     base_url = "https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/"
#     save_path = "./data/git/"

#     for file in files:
#         response = requests.get(base_url + file)
#         if response.status_code == 200:
#             with open(os.path.join(save_path, file), 'wb') as f:
#                 f.write(response.content)
#         else:
#             print(f"Failed to download {file}")

# # Execute file download
# download_github_files()


---

# Sherdog
##### [sherdog.com](https://www.sherdog.com)

In [3]:
# Sherdog: Event URLs
# event_urls_sherdog.csv

urls = [
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/1',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/2',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/3',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/4',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/5',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/6',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/7',
    'https://www.sherdog.com/organizations/Ultimate-Fighting-Championship-UFC-2/recent-events/8'
]
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/121.0"
]
headers = {"User-Agent": random.choice(user_agents)}
df = pd.read_csv('./data/event_urls_sherdog.csv') if os.path.isfile('./data/event_urls_sherdog.csv') else pd.DataFrame(columns=['Event_URL'])
existing_urls = set(df['Event_URL'])

print(f"Starting to scrape {len(urls)} URLs...")
for i, url in enumerate(urls, 1):
    print(f"Processing URL {i}/{len(urls)}")
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_div = soup.find('div', {'class': 'single_tab', 'id': 'recent_tab'})
        if specific_div:
            new_urls = [a.get('href') for a in specific_div.find_all('a', itemprop='url') if a.get('href') and a.get('href') not in existing_urls]
            df = pd.concat([df, pd.DataFrame(new_urls, columns=['Event_URL'])], ignore_index=True)
    except requests.RequestException:
        print(f"Failed to process URL {i}")
        pass

df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Updated event URLs saved.")

# Remove Broken URL's 
urls_to_delete = {
    "/events/UFC-233-Ultimate-Fighting-Championship-233-72021",
    "/events/UFC-Fight-Night-97-Lamas-vs-Penn-90890",
    "/events/UFC-176-Aldo-vs-Mendes-2-37609",
    "/events/UFC-151-Jones-vs-Henderson-25809"
}
df = pd.read_csv("./data/event_urls_sherdog.csv")
df = df[~df["Event_URL"].isin(urls_to_delete)]
df.to_csv("./data/event_urls_sherdog.csv", index=False)
print("Broken URLs removed.")

# Event URLs 
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
]
df = pd.read_csv('./data/event_urls_sherdog.csv')
df['Event_URL'] = "https://sherdog.com" + df['Event_URL'].astype(str)
event_dates = []
with requests.Session() as session:
    session.headers.update({"User-Agent": random.choice(user_agents)})
    with ThreadPoolExecutor(max_workers=40) as executor:
        # futures = {executor.submit(session.get, url, {"User-Agent": random.choice(user_agents)}, 30): url for url in df['Event_URL']}
        futures = {executor.submit(lambda url: session.get(url, headers={"User-Agent": random.choice(user_agents)}, timeout=30), url): url for url in df['Event_URL']}
        for future in tqdm(as_completed(futures), total=len(df['Event_URL']), desc="Scraping Events", unit="Event"):
            url = futures[future]
            try:
                response = future.result()
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    event_date_meta = soup.find('meta', itemprop='startDate')
                    event_dates.append(event_date_meta['content'].strip() if event_date_meta else None)
                else:
                    event_dates.append("Error")
            except requests.exceptions.RequestException:
                event_dates.append("Error")
df['Event Date'] = event_dates
df.to_csv('./data/event_urls_sherdog.csv', index=False)
print("Event dates appended successfully.")
print(f"Total number of rows including the header in event_urls_sherdog.csv: {len(df)}")
print(f"Column names: {list(df.columns)}")


Starting to scrape 8 URLs...
Processing URL 1/8
Processing URL 2/8
Processing URL 3/8
Processing URL 4/8
Processing URL 5/8
Processing URL 6/8
Processing URL 7/8
Processing URL 8/8
Updated event URLs saved.
Broken URLs removed.


Scraping Events:   0%|          | 0/741 [00:00<?, ?Event/s]

Event dates appended successfully.
Total number of rows including the header in event_urls_sherdog.csv: 741
Column names: ['Event_URL', 'Event Date']


In [4]:
# Sherdog: Event Data
# event_data_sherdog.csv

warnings.filterwarnings("ignore", category=FutureWarning)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}
urls_df = pd.read_csv('data/event_urls_sherdog.csv')
all_data = []
def fetch_event_data(url, session):
    full_url = f'https://sherdog.com{url}' if not url.startswith('http') else url
    event_data = []
    try:
        with session.get(full_url, headers=headers) as response:
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                event_name = soup.find('span', itemprop='name').text.strip()
                event_location = soup.find('span', itemprop='location').text.strip()
                event_date = soup.find('meta', itemprop='startDate')['content'].strip()
                main_event_fighters = soup.find_all('div', class_='fighter')
                if main_event_fighters:
                    fighter1 = main_event_fighters[0].find('span', itemprop='name').text.strip()
                    fighter2 = main_event_fighters[1].find('span', itemprop='name').text.strip()
                    fighter1_id = main_event_fighters[0].find('a', itemprop='url')['href'].split('-')[-1]
                    fighter2_id = main_event_fighters[1].find('a', itemprop='url')['href'].split('-')[-1]
                    weight_class = soup.find('span', class_='weight_class').text.strip()
                    winning_fighter = fighter1  
                    winning_method_em = soup.find('em', string='Method').parent
                    winning_method = winning_method_em.contents[2].strip()
                    winning_round_em = soup.find('em', string='Round').parent
                    winning_round = winning_round_em.contents[2].strip()
                    winning_time_em = soup.find('em', string='Time').parent
                    winning_time = winning_time_em.contents[2].strip()
                    referee_em = soup.find('em', string='Referee').parent
                    referee = referee_em.find('a').text.strip()
                    event_data.append({
                        'Event Name': event_name,
                        'Event Location': event_location,
                        'Event Date': event_date,
                        'Fighter 1': fighter1,
                        'Fighter 2': fighter2,
                        'Fighter 1 ID': fighter1_id,
                        'Fighter 2 ID': fighter2_id,
                        'Weight Class': weight_class,
                        'Winning Fighter': winning_fighter,
                        'Winning Method': winning_method,
                        'Winning Round': winning_round,
                        'Winning Time': winning_time,
                        'Referee': referee,
                        'Fight Type': 'Main Event'
                    })
                other_bouts = soup.find_all('tr', itemprop='subEvent')
                for bout in other_bouts:
                    fighters = bout.find_all('div', class_='fighter_list')
                    if len(fighters) >= 2:
                        fighter1 = fighters[0].find('img')['title']
                        fighter2 = fighters[1].find('img')['title']
                        fighter1_url = fighters[0].find('a', itemprop='url')['href']
                        fighter2_url = fighters[1].find('a', itemprop='url')['href']
                        fighter1_id = fighter1_url.split('-')[-1]
                        fighter2_id = fighter2_url.split('-')[-1]
                        weight_class = bout.find('span', class_='weight_class')
                        weight_class = weight_class.text.strip() if weight_class else "Unknown"
                        winning_method = bout.find('td', class_='winby').find('b').get_text(strip=True)
                        winning_round = bout.find_all('td')[-2].get_text(strip=True)
                        winning_time = bout.find_all('td')[-1].get_text(strip=True)
                        referee = bout.find('td', class_='winby').find('a').get_text(strip=True)
                        event_data.append({
                            'Event Name': event_name,
                            'Event Location': event_location,
                            'Event Date': event_date,
                            'Fighter 1': fighter1,
                            'Fighter 2': fighter2,
                            'Fighter 1 ID': fighter1_id,
                            'Fighter 2 ID': fighter2_id,
                            'Weight Class': weight_class,
                            'Winning Fighter': fighter1,  
                            'Winning Method': winning_method,
                            'Winning Round': winning_round,
                            'Winning Time': winning_time,
                            'Referee': referee,
                            'Fight Type': 'Undercard'
                        })
        return event_data
    except Exception as e:
        print(f"Request failed for {full_url}: {e}")
        return None
session = requests.Session()
total_urls = len(urls_df['Event_URL'])
completed_requests = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    futures = [executor.submit(fetch_event_data, url, session) for url in urls_df['Event_URL']]
    for future in concurrent.futures.as_completed(futures):
        data = future.result()
        completed_requests += 1
        progress_percentage = (completed_requests / total_urls) * 100
        print(f"Completed {completed_requests}/{total_urls} requests ({progress_percentage:.2f}%)")
        if data:
            all_data.extend(data)
df = pd.DataFrame(all_data)
file_path = './data/event_data_sherdog.csv'
write_mode = 'a' if os.path.isfile(file_path) else 'w'
df.to_csv(file_path, mode=write_mode, header=not os.path.isfile(file_path), index=False)
print("Data successfully written to data/event_data_sherdog.csv")
print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv(file_path, index=False)

Completed 1/741 requests (0.13%)
Completed 2/741 requests (0.27%)
Completed 3/741 requests (0.40%)
Completed 4/741 requests (0.54%)
Completed 5/741 requests (0.67%)
Completed 6/741 requests (0.81%)
Completed 7/741 requests (0.94%)
Completed 8/741 requests (1.08%)
Completed 9/741 requests (1.21%)
Completed 10/741 requests (1.35%)
Completed 11/741 requests (1.48%)
Completed 12/741 requests (1.62%)
Completed 13/741 requests (1.75%)
Completed 14/741 requests (1.89%)
Completed 15/741 requests (2.02%)
Completed 16/741 requests (2.16%)
Completed 17/741 requests (2.29%)
Completed 18/741 requests (2.43%)
Completed 19/741 requests (2.56%)
Completed 20/741 requests (2.70%)
Completed 21/741 requests (2.83%)
Completed 22/741 requests (2.97%)
Completed 23/741 requests (3.10%)
Completed 24/741 requests (3.24%)
Completed 25/741 requests (3.37%)
Completed 26/741 requests (3.51%)
Completed 27/741 requests (3.64%)
Completed 28/741 requests (3.78%)
Completed 29/741 requests (3.91%)
Completed 30/741 reques

In [5]:
# Sherdog: Fighter IDs
# fighter_id_sherdog.csv

df = pd.read_csv('./data/event_data_sherdog.csv')
df2 = pd.DataFrame(columns=['Fighter', 'Fighter_ID'])
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
for index, row in df.iterrows():
    fighter1 = row['Fighter 1']
    fighter2 = row['Fighter 2']
    fighter1_id = row['Fighter 1 ID']
    fighter2_id = row['Fighter 2 ID']
    for fighter, fighter_id in zip([fighter1, fighter2], [fighter1_id, fighter2_id]):
        if fighter not in df2['Fighter'].values and fighter_id not in df2['Fighter_ID'].values:
            df2 = pd.concat([df2, pd.DataFrame([{'Fighter': fighter, 'Fighter_ID': fighter_id}])])  # adjusted line
df2.to_csv('./data/fighter_id_sherdog.csv', index=False)
print("Data successfully written to data/fighter_id_sherdog.csv")
print(f"Total number of rows: {len(df2)}")
print(f"Column names: {list(df2.columns)}")

# Remove nicknames
df = pd.read_csv('./data/fighter_id_sherdog.csv')
df['Fighter'] = df['Fighter'].str.replace(r" '.+?'", "", regex=True)
df.to_csv('./data/fighter_id_sherdog.csv', index=False)
df = pd.read_csv('./data/event_data_sherdog.csv')
for col in ['Fighter 1', 'Fighter 2', 'Winning Fighter']:
    df[col] = df[col].str.replace(r" '.+?'", "", regex=True)
df.to_csv('./data/event_data_sherdog.csv', index=False)

# Add 'UFC' indicator to current fighters in fighter_id_sherdog.csv.csv
df = pd.read_csv('./data/fighter_id_sherdog.csv')
df['UFC'] = 'y'
df.to_csv('./data/fighter_id_sherdog.csv', index=False)
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv('./data/fighter_id_sherdog.csv', index=False)


Data successfully written to data/fighter_id_sherdog.csv
Total number of rows: 2650
Column names: ['Fighter', 'Fighter_ID']


In [6]:
# Sherdog: Fighter Info
# fighter_info.csv

def scrape_fighter_general_info_sherdog(fighter, fighter_id):
    url = f'https://www.sherdog.com/fighter/{fighter_id}'
    headers = {"User-Agent": "'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'"}
    
    # Add retry logic
    max_retries = 6  # Try for up to 1 minute (6 * 10 seconds)
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                break
        except (requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
            retry_count += 1
            if retry_count == max_retries:
                print(f"Failed to fetch data for fighter {fighter} after {max_retries} retries")
                return {}
            print(f"Connection error, retrying in 10 seconds... (Attempt {retry_count}/{max_retries})")
            time.sleep(10)
            continue
            
    if response.status_code != 200:
        return {}
        
    soup = BeautifulSoup(response.content, 'html.parser')
    fighter_dict = {}
    try:
        fighter_data = soup.find('div', class_='fighter-data')
    except AttributeError:
        fighter_data = None
    try:
        birthdate = soup.find('span', itemprop='birthDate')
        birthdate = (birthdate.text).strip('""')
    except AttributeError:
        birthdate = '-'
    try:
        nationality = soup.find('strong', itemprop='nationality')
        nationality = (nationality.text).strip()
    except AttributeError:
        nationality = '-'
    try:
        hometown = soup.find('span', {'itemprop': 'addressLocality'}).text
        hometown = hometown.strip()
    except AttributeError:
        hometown = '-'
    try:
        association = soup.find('span', {'itemprop': 'name'}).text
        association = association.strip()
    except AttributeError:
        association = '-'
    try:
        weight_class_div = fighter_data.find('div', {'class': 'association-class'})
        links = weight_class_div.find_all('a')
        weight_class = links[-1].text
        weight_class = weight_class.strip()
    except (AttributeError, IndexError):
        weight_class = ''
    try:
        nickname = soup.find('span', class_='nickname')
        nickname = (nickname.text).strip('"')
    except AttributeError:
        nickname = '-'
    try:
        height = soup.find('b', itemprop='height')
        height = (height.text).strip('"')
    except AttributeError:
        height = '-'
    try:
        wins = soup.find('div', class_='winloses win').find_all('span')[1]
        wins = (wins.text).strip()
    except AttributeError:
        wins = '-'
    try:
        losses = soup.find('div', class_='winloses lose').find_all('span')[1]
        losses = (losses.text).strip()
    except AttributeError:
        losses = '-'
    dec_data_list = []
    try:
        win_type = fighter_data.find_all('div', class_='meter-title', string='DECISIONS')
        for method in win_type:
            if method.text.startswith('DECISIONS'):
                dec_data = method.find_next('div', class_='pl').text
                dec_data_list.append(dec_data)
        wins_dec = (dec_data_list[0]).strip()
        losses_dec = (dec_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_dec = '-'
        losses_dec = '-'
    ko_data_list = []
    try:
        win_type = soup.find_all('div', class_='meter-title')
        for method in win_type:
            if method.text.startswith('KO'):
                ko_data = method.find_next('div', class_='pl').text
                ko_data_list.append(ko_data)
        wins_ko = (ko_data_list[0]).strip()
        losses_ko = (ko_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_ko = '-'
        losses_ko = '-'
    sub_data_list = []
    try:
        win_type = fighter_data.find_all('div', class_='meter-title', string='SUBMISSIONS')
        for method in win_type:
            if method.text.startswith('SUBMISSIONS'):
                sub_data = method.find_next('div', class_='pl').text
                sub_data_list.append(sub_data)
        wins_sub = (sub_data_list[0]).strip()
        losses_sub = (sub_data_list[1]).strip()
    except (AttributeError, IndexError):
        wins_sub = '-'
        losses_sub = '-'
    fighter_dict = {
        'Fighter': fighter,
        'Nickname': nickname,
        'Birth Date': birthdate,
        'Nationality': nationality,
        'Hometown': hometown,
        'Association': association,
        'Weight Class': weight_class,
        'Height': height,
        'Wins': wins,
        'Losses': losses,
        'Win_Decision': wins_dec,
        'Win_KO': wins_ko,
        'Win_Sub': wins_sub,
        'Loss_Decision': losses_dec,
        'Loss_KO': losses_ko,
        'Loss_Sub': losses_sub,
        'Fighter_ID': fighter_id
    }
    return fighter_dict

def scrape_fighters_concurrently():
    warnings.filterwarnings("ignore", category=FutureWarning)
    df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
    fighter_data_list = []
    total_fighters = len(df_fighter_id)
    fighters_processed = 0
    with ThreadPoolExecutor(max_workers=40) as executor:
        future_to_fighter = {executor.submit(scrape_fighter_general_info_sherdog, row['Fighter'], row['Fighter_ID']): row for index, row in df_fighter_id.iterrows()}
        for future in tqdm(as_completed(future_to_fighter), total=len(future_to_fighter)):
            fighter_data = future.result()
            if fighter_data:
                fighter_data_list.append(fighter_data)
            fighters_processed += 1
            print(f"Scraping Fighter Info: {fighters_processed}/{total_fighters} fighters processed")
    new_df = pd.DataFrame(fighter_data_list)
    new_df.to_csv('./data/fighter_info.csv', index=False)
scrape_fighters_concurrently()

df = pd.read_csv('./data/fighter_info.csv')
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df.to_csv('./data/fighter_info.csv', index=False)
print("Data successfully written to data/fighter_info.csv")
print(f"Total number of rows: {len(df)}")
print(f"Column names: {list(df.columns)}")


  0%|          | 0/2650 [00:00<?, ?it/s]

Scraping Fighter Info: 1/2650 fighters processed
Scraping Fighter Info: 2/2650 fighters processed
Scraping Fighter Info: 3/2650 fighters processed
Scraping Fighter Info: 4/2650 fighters processed
Scraping Fighter Info: 5/2650 fighters processed
Scraping Fighter Info: 6/2650 fighters processed
Scraping Fighter Info: 7/2650 fighters processed
Scraping Fighter Info: 8/2650 fighters processed
Scraping Fighter Info: 9/2650 fighters processed
Scraping Fighter Info: 10/2650 fighters processed
Scraping Fighter Info: 11/2650 fighters processed
Scraping Fighter Info: 12/2650 fighters processed
Scraping Fighter Info: 13/2650 fighters processed
Scraping Fighter Info: 14/2650 fighters processed
Scraping Fighter Info: 15/2650 fighters processed
Scraping Fighter Info: 16/2650 fighters processed
Scraping Fighter Info: 17/2650 fighters processed
Scraping Fighter Info: 18/2650 fighters processed
Scraping Fighter Info: 19/2650 fighters processed
Scraping Fighter Info: 20/2650 fighters processed
Scraping 

In [7]:
# # # # # Sherdog: Fighters
# # # # # data/fighters/*

# # # # os.makedirs('./data/fighters/', exist_ok=True)
# # # # session = requests.Session()
# # # # session.headers.update({
# # # #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
# # # # })
# # # # def scrape_fighter_fights_sherdog(session, fighter_name, fighter_id, fighter_url):
# # # #     try:
# # # #         response = session.get(fighter_url, timeout=30)
# # # #         if response.status_code == 200:
# # # #             soup = BeautifulSoup(response.content, 'html.parser')
# # # #             table = soup.find('table', {'class': 'new_table fighter'})
# # # #             rows = table.find_all('tr')[1:] if table else []
# # # #             fight_data = []
# # # #             new_opponents = []
# # # #             for row in rows:
# # # #                 cols = row.find_all('td')
# # # #                 fight_dict = {
# # # #                     'Result': cols[0].text.strip(),
# # # #                     'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
# # # #                     'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
# # # #                     'Method/Referee': cols[3].text.strip().split('\n')[0],
# # # #                     'Rounds': cols[4].text.strip(),
# # # #                     'Time': cols[5].text.strip()
# # # #                 }
# # # #                 fight_data.append(fight_dict)
# # # #                 opponent_link = cols[1].find('a')['href'] if cols[1].find('a') else None
# # # #                 if opponent_link:
# # # #                     opponent_id = opponent_link.split('-')[-1]
# # # #                     new_opponents.append({'Fighter': fight_dict['Opponent'], 'Fighter_ID': opponent_id})
# # # #             return fighter_id, fighter_name, fight_data, new_opponents
# # # #         print(f"Failed to retrieve page for {fighter_name} (Status code: {response.status_code})")
# # # #     except requests.exceptions.RequestException as e:
# # # #         print(f"Request error for {fighter_name}: {e}")
# # # #     return fighter_id, fighter_name, [], []
# # # # df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
# # # # all_new_opponents = []
# # # # def process_fighter(row, session):
# # # #     fighter_name = row['Fighter']
# # # #     fighter_id = row['Fighter_ID']
# # # #     fighter_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
# # # #     if os.path.exists(fighter_file):
# # # #         print(f"Skipping {fighter_name} - file already exists")
# # # #         return fighter_id, fighter_name, [], []
        
# # # #     fighter_url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
# # # #     return scrape_fighter_fights_sherdog(session, fighter_name, fighter_id, fighter_url)

# # # # total_fighters = len(df_fighter_id)
# # # # fighters_processed = 0
# # # # print(f"\nStarting to process {total_fighters} fighters")

# # # # with ThreadPoolExecutor(max_workers=10) as executor:
# # # #     futures = [executor.submit(process_fighter, row, session) for _, row in df_fighter_id.iterrows()]
# # # #     for future in as_completed(futures):
# # # #         fighter_id, fighter_name, fight_data, new_opponents = future.result()
# # # #         fighters_processed += 1
# # # #         print(f"Progress: {fighters_processed}/{total_fighters} fighters processed ({(fighters_processed/total_fighters)*100:.1f}%)")
        
# # # #         if fight_data:
# # # #             output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
# # # #             pd.DataFrame(fight_data).to_csv(output_file, index=False)
# # # #             print(f"Saved {len(fight_data)} fights for {fighter_name} to {output_file}")
# # # #             all_new_opponents.extend(new_opponents)

# # # # if all_new_opponents:
# # # #     print(f"\nFound {len(all_new_opponents)} new opponents")
# # # #     df_new_opponents = pd.DataFrame(all_new_opponents).drop_duplicates()
# # # #     df_fighter_id = pd.concat([df_fighter_id, df_new_opponents], ignore_index=True).drop_duplicates()
# # # #     df_fighter_id.to_csv('./data/fighter_id_sherdog.csv', index=False)
# # # #     print(f"Updated fighter_id_sherdog.csv with {len(df_new_opponents)} new unique opponents")


# # # # import os
# # # # import requests
# # # # import pandas as pd
# # # # import time
# # # # from bs4 import BeautifulSoup
# # # # from concurrent.futures import ThreadPoolExecutor, as_completed
# # # # from tqdm import tqdm

# # # # Ensure directory exists
# # # os.makedirs('./data/fighters/', exist_ok=True)

# # # def scrape_fighter_fights_sherdog(fighter_name, fighter_id, session):
# # #     """Scrapes fight history of a fighter from Sherdog with retry logic."""
# # #     fighter_url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
# # #     max_retries = 6
# # #     retry_count = 0
# # #     while retry_count < max_retries:
# # #         try:
# # #             response = session.get(fighter_url, timeout=30)
# # #             if response.status_code == 200:
# # #                 break
# # #         except (requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
# # #             retry_count += 1
# # #             if retry_count == max_retries:
# # #                 print(f"Failed to fetch data for fighter {fighter_name} after {max_retries} retries")
# # #                 return fighter_id, fighter_name, []
# # #             print(f"Connection error, retrying in 10 seconds... (Attempt {retry_count}/{max_retries})")
# # #             time.sleep(10)
# # #             continue
    
# # #     if response.status_code != 200:
# # #         return fighter_id, fighter_name, []
    
# # #     soup = BeautifulSoup(response.content, 'html.parser')
# # #     table = soup.find('table', {'class': 'new_table fighter'})
# # #     rows = table.find_all('tr')[1:] if table else []
# # #     fight_data = []
    
# # #     for row in rows:
# # #         cols = row.find_all('td')
# # #         fight_dict = {
# # #             'Result': cols[0].text.strip(),
# # #             'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
# # #             'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
# # #             'Method/Referee': cols[3].text.strip().split('\n')[0],
# # #             'Rounds': cols[4].text.strip(),
# # #             'Time': cols[5].text.strip()
# # #         }
# # #         fight_data.append(fight_dict)
    
# # #     return fighter_id, fighter_name, fight_data

# # # df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')

# # # def process_fighter(row):
# # #     """Processes a single fighter's fight history with a per-thread session."""
# # #     with requests.Session() as session:
# # #         session.headers.update({
# # #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# # #         })
# # #         fighter_name = row['Fighter']
# # #         fighter_id = row['Fighter_ID']
# # #         fighter_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
# # #         if os.path.exists(fighter_file):
# # #             print(f"Skipping {fighter_name} - file already exists")
# # #             return fighter_id, fighter_name, []
# # #         return scrape_fighter_fights_sherdog(fighter_name, fighter_id, session)

# # # total_fighters = len(df_fighter_id)
# # # fighters_processed = 0
# # # print(f"\nStarting to process {total_fighters} fighters")

# # # fighter_data_list = []
# # # with ThreadPoolExecutor(max_workers=10) as executor:
# # #     future_to_fighter = {executor.submit(process_fighter, row): row for _, row in df_fighter_id.iterrows()}
    
# # #     for future in tqdm(as_completed(future_to_fighter), total=len(future_to_fighter)):
# # #         fighter_id, fighter_name, fight_data = future.result()
# # #         fighters_processed += 1
# # # #         print(f"Scraping Fighter Fights: {fighters_processed}/{total_fighters} fighters processed")
        
# # #         if fight_data:
# # #             output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
# # #             pd.DataFrame(fight_data).to_csv(output_file, index=False)
# # # #             print(f"Saved {len(fight_data)} fights for {fighter_name} to {output_file}")
# # import os
# # import requests
# # import pandas as pd
# # import time
# # from bs4 import BeautifulSoup
# # from concurrent.futures import ThreadPoolExecutor, as_completed
# # from tqdm.notebook import tqdm

# # # Ensure directory exists
# # os.makedirs('./data/fighters/', exist_ok=True)

# # # Initialize session with headers
# # session = requests.Session()
# # session.headers.update({
# #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# # })

# # def scrape_fighter_fights_sherdog(fighter_name, fighter_id, session):
# #     """Scrapes fight history of a fighter from Sherdog with retry logic."""
# #     fighter_url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
# #     max_retries = 6
# #     retry_count = 0
# #     while retry_count < max_retries:
# #         try:
# #             response = session.get(fighter_url, timeout=30)
# #             if response.status_code == 200:
# #                 break
# #         except (requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
# #             retry_count += 1
# #             if retry_count == max_retries:
# #                 print(f"Failed to fetch data for fighter {fighter_name} after {max_retries} retries")
# #                 return fighter_id, fighter_name, []
# #             print(f"Connection error, retrying in 10 seconds... (Attempt {retry_count}/{max_retries})")
# #             time.sleep(7)
# #             continue
    
# #     if response.status_code != 200:
# #         return fighter_id, fighter_name, []
    
# #     soup = BeautifulSoup(response.content, 'html.parser')
# #     table = soup.find('table', {'class': 'new_table fighter'})
# #     rows = table.find_all('tr')[1:] if table else []
# #     fight_data = []
    
# #     for row in rows:
# #         cols = row.find_all('td')
# #         fight_dict = {
# #             'Result': cols[0].text.strip(),
# #             'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
# #             'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
# #             'Method/Referee': cols[3].text.strip().split('\n')[0],
# #             'Rounds': cols[4].text.strip(),
# #             'Time': cols[5].text.strip()
# #         }
# #         fight_data.append(fight_dict)
    
# #     return fighter_id, fighter_name, fight_data

# # df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')

# # def process_fighter(row, processed_count):
# #     """Processes a single fighter's fight history with a per-thread session."""
# #     fighter_name = row['Fighter']
# #     fighter_id = row['Fighter_ID']
# #     fighter_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
    
# #     # Skip if the file already exists
# #     if os.path.exists(fighter_file):
# #         print(f"Skipping {fighter_name} - file already exists")
# #         return fighter_id, fighter_name, []

# #     # Sleep every 100 fighters processed to help with rate limiting
# #     if processed_count % 100 == 0 and processed_count > 0:
# #         print(f"Processed {processed_count} fighters. Sleeping for 5 seconds...")
# #         time.sleep(5)  # 5-second delay after every 100 fighters
    
# #     return scrape_fighter_fights_sherdog(fighter_name, fighter_id, session)

# # total_fighters = len(df_fighter_id)
# # fighters_processed = 0
# # print(f"\nStarting to process {total_fighters} fighters")

# # fighter_data_list = []

# # # Using ThreadPoolExecutor with multiple threads
# # with ThreadPoolExecutor(max_workers=5) as executor:
# #     future_to_fighter = {executor.submit(process_fighter, row, fighters_processed): row for _, row in df_fighter_id.iterrows()}
    
# #     # Processing the results as they complete
# #     for future in tqdm(as_completed(future_to_fighter), total=len(future_to_fighter)):
# #         fighter_id, fighter_name, fight_data = future.result()
# #         fighters_processed += 1
# #         # print(f"Scraping Fighter Fights: {fighters_processed}/{total_fighters} fighters processed")
        
# #         # Save the data
# #         if fight_data:
# #             output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
# #             pd.DataFrame(fight_data).to_csv(output_file, index=False)
# #             # print(f"Saved {len(fight_data)} fights for {fighter_name} to {output_file}")

# import os
# import requests
# import pandas as pd
# import time
# from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm.notebook import tqdm

# # Ensure directory exists
# os.makedirs('./data/fighters/', exist_ok=True)

# # Initialize session with headers
# session = requests.Session()
# session.headers.update({
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# })

# def scrape_fighter_fights_sherdog(fighter_name, fighter_id, session):
#     """Scrapes fight history of a fighter from Sherdog with retry logic."""
#     fighter_url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
#     max_retries = 6
#     retry_count = 0
#     while retry_count < max_retries:
#         try:
#             response = session.get(fighter_url, timeout=30)
#             if response.status_code == 200:
#                 break
#         except (requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
#             retry_count += 1
#             if retry_count == max_retries:
#                 print(f"Failed to fetch data for fighter {fighter_name} after {max_retries} retries")
#                 return fighter_id, fighter_name, []
#             print(f"Connection error, retrying in 10 seconds... (Attempt {retry_count}/{max_retries})")
#             time.sleep(7)
#             continue
    
#     if response.status_code != 200:
#         return fighter_id, fighter_name, []
    
#     soup = BeautifulSoup(response.content, 'html.parser')
#     table = soup.find('table', {'class': 'new_table fighter'})
#     rows = table.find_all('tr')[1:] if table else []
#     fight_data = []
    
#     for row in rows:
#         cols = row.find_all('td')
#         fight_dict = {
#             'Result': cols[0].text.strip(),
#             'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
#             'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
#             'Method/Referee': cols[3].text.strip().split('\n')[0],
#             'Rounds': cols[4].text.strip(),
#             'Time': cols[5].text.strip()
#         }
#         fight_data.append(fight_dict)
    
#     return fighter_id, fighter_name, fight_data

# df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')

# def process_fighter(row, processed_count):
#     """Processes a single fighter's fight history with a per-thread session."""
#     fighter_name = row['Fighter']
#     fighter_id = row['Fighter_ID']
#     fighter_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
    
#     # Skip if the file already exists
#     if os.path.exists(fighter_file):
#         print(f"Skipping {fighter_name} - file already exists")
#         return fighter_id, fighter_name, []

#     # Sleep every 100 fighters processed to help with rate limiting
#     if processed_count % 100 == 0 and processed_count > 0:
#         print(f"Processed {processed_count} fighters. Sleeping for 5 seconds...")
#         time.sleep(5)  # 5-second delay after every 100 fighters
    
#     return scrape_fighter_fights_sherdog(fighter_name, fighter_id, session)

# total_fighters = len(df_fighter_id)
# fighters_processed = 0
# print(f"\nStarting to process {total_fighters} fighters")

# fighter_data_list = []

# # Using ThreadPoolExecutor with multiple threads
# with ThreadPoolExecutor(max_workers=20) as executor:
#     future_to_fighter = {executor.submit(process_fighter, row, fighters_processed): row for _, row in df_fighter_id.iterrows()}
    
#     # Processing the results as they complete
#     for future in tqdm(as_completed(future_to_fighter), total=len(future_to_fighter)):
#         fighter_id, fighter_name, fight_data = future.result()
#         fighters_processed += 1
#         # print(f"Scraping Fighter Fights: {fighters_processed}/{total_fighters} fighters processed")
        
#         # Save the data
#         if fight_data:
#             output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
#             pd.DataFrame(fight_data).to_csv(output_file, index=False)
#             # print(f"Saved {len(fight_data)} fights for {fighter_name} to {output_file}")


In [8]:
# Sherdog: Fighters
# data/fighters/*
# Asyncio Version ^

nest_asyncio.apply()
os.makedirs('./data/fighters/', exist_ok=True)
df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
}
async def fetch_fighter_data(session, fighter_name, fighter_id):
    """Asynchronously fetch fighter data from Sherdog"""
    url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
    for _ in range(6):  
        try:
            async with session.get(url, headers=HEADERS, timeout=aiohttp.ClientTimeout(total=60), raise_for_status=True) as response:
                return fighter_name, fighter_id, await response.text()
        except aiohttp.ClientResponseError as e:
            if e.status == 429:
                print(f"Rate limited on {fighter_name}. Sleeping for 10 sec...")
                await asyncio.sleep(10)
            else:
                print(f"HTTP error {e.status} on {fighter_name}. Retrying...")
        except asyncio.TimeoutError:
            print(f"Timeout error on {fighter_name}. Retrying...")
        except aiohttp.ClientError as e:
            print(f"Request error for {fighter_name}: {e}")
        await asyncio.sleep(11)  
    return fighter_name, fighter_id, None  
async def parse_fighter_data(session, fighter_name, fighter_id):
    """Extract fight data from Sherdog"""
    _, _, html = await fetch_fighter_data(session, fighter_name, fighter_id)
    if not html:
        return None
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', {'class': 'new_table fighter'})
    rows = table.find_all('tr')[1:] if table else []
    fight_data = []
    for row in rows:
        cols = row.find_all('td')
        fight_dict = {
            'Result': cols[0].text.strip(),
            'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
            'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
            'Method/Referee': cols[3].text.strip().split('\n')[0],
            'Rounds': cols[4].text.strip(),
            'Time': cols[5].text.strip()
        }
        fight_data.append(fight_dict)
    return fighter_name, fighter_id, fight_data
async def main():
    """Run all requests asynchronously"""
    connector = aiohttp.TCPConnector(limit=10)
    async with aiohttp.ClientSession() as session:
        tasks = [
            parse_fighter_data(session, row['Fighter'], row['Fighter_ID'])
            for _, row in df_fighter_id.iterrows()
        ]
        results = await tqdm_async.gather(*tasks)
    for fighter_name, fighter_id, fight_data in results:
        if fight_data:
            output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
            pd.DataFrame(fight_data).to_csv(output_file, index=False)
asyncio.get_event_loop().run_until_complete(main())


# Try ^

# import os
# import aiohttp
# import asyncio
# import pandas as pd
# from bs4 import BeautifulSoup
# import nest_asyncio
# from tqdm.asyncio import tqdm
# nest_asyncio.apply()
# os.makedirs('./data/fighters/', exist_ok=True)
# df_fighter_id = pd.read_csv('./data/fighter_id_sherdog.csv')
# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
# }

# async def fetch_fighter_data(session, fighter_name, fighter_id):
#     url = f"https://www.sherdog.com/fighter/{fighter_name.replace(' ', '-')}-{fighter_id}"
#     retries = 6
#     delay = 5
#     for attempt in range(retries):
#         try:
#             async with session.get(url, headers=HEADERS, timeout=60) as response:
#                 if response.status == 200:
#                     return fighter_name, fighter_id, await response.text()
#                 elif response.status == 429:
#                     print(f"Rate limited on {fighter_name}. Sleeping for 10 sec...")
#                     await asyncio.sleep(10)
#         except (aiohttp.ClientError, asyncio.TimeoutError) as e:
#             print(f"Request error for {fighter_name}: {e}")
#         print(f"Retrying {fighter_name} in {delay} seconds...")
#         await asyncio.sleep(delay)
#         delay *= 2
#     return fighter_name, fighter_id, None

# async def parse_fighter_data(session, fighter_name, fighter_id):
#     _, _, html = await fetch_fighter_data(session, fighter_name, fighter_id)
#     if not html:
#         return None
#     soup = BeautifulSoup(html, 'html.parser')
#     table = soup.find('table', {'class': 'new_table fighter'})
#     rows = table.find_all('tr')[1:] if table else []
#     fight_data = []
#     for row in rows:
#         cols = row.find_all('td')
#         fight_dict = {
#             'Result': cols[0].text.strip(),
#             'Opponent': cols[1].find('a').text.strip() if cols[1].find('a') else '-',
#             'Event Date': cols[2].find_all('span')[-1].text.strip() if cols[2].find_all('span') else '-',
#             'Method/Referee': cols[3].text.strip().split('\n')[0],
#             'Rounds': cols[4].text.strip(),
#             'Time': cols[5].text.strip()
#         }
#         fight_data.append(fight_dict)
#     return fighter_name, fighter_id, fight_data

# async def main():
#     connector = aiohttp.TCPConnector(limit=40)
#     async with aiohttp.ClientSession() as session:
#         tasks = [
#             parse_fighter_data(session, row['Fighter'], row['Fighter_ID'])
#             for _, row in df_fighter_id.iterrows()
#         ]
#         results = await tqdm.gather(*tasks)
#     for fighter_name, fighter_id, fight_data in results:
#         if fight_data:
#             output_file = f"./data/fighters/{fighter_name.replace(' ', '_')}_{fighter_id}.csv"
#             pd.DataFrame(fight_data).to_csv(output_file, index=False)
#             print(f"Saved {len(fight_data)} fights for {fighter_name} to {output_file}")

# asyncio.get_event_loop().run_until_complete(main())


 27%|██▋       | 708/2650 [00:53<01:49, 17.80it/s]

Timeout error on Leonard Garcia. Retrying...
Timeout error on Courtney Turner. Retrying...
Timeout error on Kamaru Usman. Retrying...
Timeout error on Alan Jouban. Retrying...
Timeout error on Martin Day. Retrying...
Timeout error on Tainara Lisboa. Retrying...
Timeout error on Muhammad Mokaev. Retrying...
Timeout error on Eugenio Tadeu. Retrying...
Timeout error on Hakeem Dawodu. Retrying...
Timeout error on Tony Halme. Retrying...
Timeout error on Ihor Potieria. Retrying...
Timeout error on Saparbek Safarov. Retrying...
Timeout error on Alistair Overeem. Retrying...
Timeout error on Jasmine Jasudavicius. Retrying...
Timeout error on Townsend Saunders. Retrying...
Timeout error on Yoel Romero. Retrying...
Timeout error on Juan Manuel Puig. Retrying...
Timeout error on Miranda Granger. Retrying...
Timeout error on Kevin Jackson. Retrying...
Timeout error on Bethe Correia. Retrying...
Timeout error on Geraldo de Freitas Jr.. Retrying...
Timeout error on Steven Koslow. Retrying...
Timeou

 54%|█████▍    | 1432/2650 [02:11<11:25,  1.78it/s]

Timeout error on Karl Williams. Retrying...
Timeout error on Andrew Holbrook. Retrying...
Timeout error on Jaqueline Amorim. Retrying...
Timeout error on Ricardo Ramos. Retrying...
Timeout error on Brendan Allen. Retrying...
Timeout error on Felipe Arantes. Retrying...
Timeout error on Carlo Pedersoli Jr.. Retrying...
Timeout error on Eric Spicely. Retrying...
Timeout error on Sergey Morozov. Retrying...
Timeout error on Rustam Khabilov. Retrying...
Timeout error on Daniel Kelly. Retrying...
Timeout error on Dominick Cruz. Retrying...
Timeout error on Steven Ray. Retrying...
Timeout error on Bradley Scott. Retrying...
Timeout error on Scott Holtzman. Retrying...
Timeout error on Aleksandra Albu. Retrying...
Timeout error on Elias Theodorou. Retrying...
Timeout error on Nikita Krylov. Retrying...
Timeout error on Danilo Belluardo. Retrying...
Timeout error on Gokhan Saki. Retrying...
Timeout error on Mike Jackson. Retrying...
Timeout error on Siyar Bahadurzada. Retrying...
Timeout error

 78%|███████▊  | 2064/2650 [03:23<00:17, 33.13it/s]

Timeout error on Claudio Silva. Retrying...
Timeout error on Mark Weir. Retrying...
Timeout error on Geoff Neal. Retrying...
Timeout error on Michinori Tanaka. Retrying...
Timeout error on Jarno Errens. Retrying...
Timeout error on Zac Pauga. Retrying...
Timeout error on Jake Matthews. Retrying...
Timeout error on Renato Verissimo. Retrying...
Timeout error on Grant Dawson. Retrying...
Timeout error on Koji Oishi. Retrying...
Timeout error on Steve Erceg. Retrying...
Timeout error on Carla Esparza. Retrying...
Timeout error on Azunna Anyanwu. Retrying...
Timeout error on Melissa Martinez. Retrying...
Timeout error on Puja Tomar. Retrying...
Timeout error on Shinsho Anzai. Retrying...
Timeout error on Tiago dos Santos e Silva. Retrying...
Timeout error on Alvin Robinson. Retrying...
Timeout error on Daniel Barrera. Retrying...
Timeout error on Victor Hugo. Retrying...
Timeout error on Fredy Serrano. Retrying...
Timeout error on Saeed Hosseini. Retrying...
Timeout error on Daniel Barez. 

100%|█████████▉| 2640/2650 [04:34<00:00, 20.29it/s]

Timeout error on Sean O'Connell. Retrying...


100%|██████████| 2650/2650 [04:47<00:00,  9.20it/s]


In [9]:
# Test

first_name = "Dustin"
last_name = "Poirier"
file_pattern = f'./data/fighters/*{first_name}*{last_name}*.csv'
matching_files = glob.glob(file_pattern)
if matching_files:
    first_file = matching_files[0]
    df = pd.read_csv(first_file)
    print(f"Total number of rows including the header in {first_file}: {len(df)}")
    print(f"Column names: {list(df.columns)}")
else:
    print("No files found matching the pattern.")


Total number of rows including the header in ./data/fighters/Dustin_Poirier_50529.csv: 40
Column names: ['Result', 'Opponent', 'Event Date', 'Method/Referee', 'Rounds', 'Time']


In [10]:
# Sherdog: Cleaning

fighter_info_df = pd.read_csv('./data/fighter_info.csv')
event_data_df = pd.read_csv('./data/event_data_sherdog.csv')
print(f"Number of rows in fighter_info.csv: {len(fighter_info_df)}")
print(f"Number of rows in event_data_sherdog.csv: {len(event_data_df)}")

# # Removing women's fights and fighters
# womens_weight_classes = ['Strawweight'] # 'Flyweight', 'Bantamweight', 'Featherweight'
# fighter_info_df = pd.read_csv('./data/fighter_info.csv')
# cleaned_fighter_info_df = fighter_info_df[~fighter_info_df['Weight Class'].isin(womens_weight_classes)]
# cleaned_fighter_info_df.to_csv('./data/fighter_info_men_only.csv', index=False)
# print(f"Cleaned dataset saved to {'./data/fighter_info_men_only.csv'}")
# event_data_df = pd.read_csv('./data/event_data_sherdog.csv')
# cleaned_event_data_df = event_data_df[~event_data_df['Weight Class'].isin(womens_weight_classes)]
# cleaned_event_data_df.to_csv('./data/event_data_sherdog_men_only.csv', index=False)
# print(f"Cleaned event dataset saved to {'./data/event_data_sherdog_men_only.csv'}")
# print(f"Number of rows in fighter_info_men_only.csv: {len(cleaned_fighter_info_df)}")
# print(f"Number of rows in event_data_sherdog_men_only.csv: {len(cleaned_event_data_df)}")

# Remove leading/trailing whitespace and normalize to lowercase for all columns
# fighter_info_df = fighter_info_df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
# event_data_df = event_data_df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
fighter_info_df = fighter_info_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
event_data_df = event_data_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove special characters for all columns
# fighter_info_df = fighter_info_df.applymap(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)
# event_data_df = event_data_df.applymap(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)

# Handle missing values
# fighter_info_df.fillna("Unknown", inplace=True)
# event_data_df.fillna("Unknown", inplace=True)

# Standardize column names
# fighter_info_df.columns = fighter_info_df.columns.str.strip().str.lower().str.replace(' ', '_')
# event_data_df.columns = event_data_df.columns.str.strip().str.lower().str.replace(' ', '_')

# Remove duplicates
print(f"Dropped {fighter_info_df.drop_duplicates(inplace=True)} duplicates from fighter_info_df")
print(f"Dropped {event_data_df.drop_duplicates(inplace=True)} duplicates from event_data_df")

# Save the cleaned datasets
fighter_info_df.to_csv('./data/fighter_info.csv', index=False)
event_data_df.to_csv('./data/event_data_sherdog.csv', index=False)
print("Data cleaned and saved successfully.")
print(f"Number of rows in fighter_info.csv: {len(fighter_info_df)}")
print(f"Number of rows in event_data_sherdog.csv: {len(event_data_df)}")


Number of rows in fighter_info.csv: 2650
Number of rows in event_data_sherdog.csv: 8143
Dropped None duplicates from fighter_info_df
Dropped None duplicates from event_data_df
Data cleaned and saved successfully.
Number of rows in fighter_info.csv: 2650
Number of rows in event_data_sherdog.csv: 8143


---

# UFC
##### [ufc.com](https://www.ufc.com)

In [11]:
# UFC: Fighters

# Sitemaps
if os.path.exists("data/ufc_roster.csv"): os.remove("data/ufc_roster.csv")
ns = {
    "s": "http://www.sitemaps.org/schemas/sitemap/0.9",
    "xhtml": "http://www.w3.org/1999/xhtml"
}
all_fighters = []
page = 1
consecutive_empty_pages = 0
max_consecutive_empty = 100
while True:
    sitemap_url = f"https://www.ufc.com/sitemap.xml?page={page}"
    try:
        response = requests.get(sitemap_url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"Page {page} does not exist. Stopping.")
        break
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {page}: {e}")
        page += 1
        continue
    try:
        root = ET.fromstring(response.content)
    except ET.ParseError as e:
        print(f"Error parsing page {page}: {e}")
        if "not well-formed (invalid token)" in str(e):
            print("Reached end of valid pages. Stopping.")
            break
        page += 1
        continue
    fighter_urls = []
    for url_elem in root.findall("s:url", ns):
        loc_elem = url_elem.find("s:loc", ns)
        if loc_elem is not None:
            url_text = loc_elem.text
            if "/athlete/" in url_text:
                fighter_urls.append(url_text)
    print(f"https://www.ufc.com/sitemap.xml?page={page} -> {len(fighter_urls)} fighter URLs")
    if len(fighter_urls) == 0:
        consecutive_empty_pages += 1
        if consecutive_empty_pages >= max_consecutive_empty:
            print(f"Found {max_consecutive_empty} consecutive empty pages. Stopping.")
            break
    else:
        consecutive_empty_pages = 0

    for url in fighter_urls:
        try:
            slug = url.split("/athlete/")[1]
            name = slug.replace("-", " ").title()
        except IndexError:
            name = ""
        all_fighters.append({"name": name, "url": url})
    page += 1
print(f"Total fighter records found: {len(all_fighters)}")
csv_filename = "data/ufc_roster.csv"
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["name", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for fighter in all_fighters:
        writer.writerow(fighter)
print(f"All fighter names and URLs saved to {csv_filename}")
df = pd.read_csv("data/ufc_roster.csv")
all_fighter_urls = df["url"].tolist()
print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")


# Fighter Profiles
roster_csv = "data/ufc_roster.csv"
df = pd.read_csv(roster_csv)
print(f"Columns in {roster_csv}: {df.columns.tolist()}")
all_fighter_urls = df["url"].dropna().tolist()
print(f"Total fighter URLs loaded from CSV: {len(all_fighter_urls)}")
profile_csv = "data/ufc_fighter_profiles.csv"
if os.path.exists("data/ufc_fighter_profiles.csv"): os.remove("data/ufc_fighter_profiles.csv")
fieldnames = ["name", "nickname", "weight_class", "profile_url", "thumbnail"]
def scrape_fighter(url):
    """
    Scrapes a fighter page and returns the extracted data as a dictionary.
    Implements basic retry logic in case of temporary rate limiting or non-200 responses.
    """
    for attempt in range(1, 6):
        try:
            resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15)
            if resp.status_code == 200:
                soup = BeautifulSoup(resp.text, "html.parser")
                name_tag = soup.find("h1", class_="hero-profile__name")
                name = name_tag.get_text(strip=True) if name_tag else "Unknown"
                nickname_tag = soup.find("p", class_="hero-profile__nickname")
                nickname = nickname_tag.get_text(strip=True) if nickname_tag else ""
                weight_class_tag = soup.find("p", class_="hero-profile__division-title")
                weight_class = weight_class_tag.get_text(strip=True) if weight_class_tag else ""
                thumbnail_tag = soup.find("img", class_="hero-profile__image")
                thumbnail = thumbnail_tag["src"] if thumbnail_tag and "src" in thumbnail_tag.attrs else ""
                fighter_data = {
                    "name": name,
                    "nickname": nickname,
                    "weight_class": weight_class,
                    "profile_url": url,
                    "thumbnail": thumbnail
                }
                return fighter_data
            elif resp.status_code == 429:
                print(f"Rate-limited. Waiting before retry... (Attempt {attempt} of 3)")
                time.sleep(60)
            else:
                print(f"Error fetching fighter page {url} (status {resp.status_code}), attempt {attempt} of 3.")
                time.sleep(30)
        except Exception as e:
            print(f"Exception occurred while scraping {url}: {e} (Attempt {attempt} of 3)")
            time.sleep(20)
    print(f"Failed to fetch fighter page after 3 retries: {url}")
    return None

results = []
max_workers = 10
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_url = {executor.submit(scrape_fighter, url): url for url in all_fighter_urls}
    for idx, future in enumerate(as_completed(future_to_url), start=1):
        data = future.result()
        if data:
            results.append(data)
        print(f"Processed {idx} of {len(all_fighter_urls)} fighter profiles")
        if idx % 100 == 0:
            with open(profile_csv, "a", newline="", encoding="utf-8") as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                for fighter_data in results:
                    writer.writerow(fighter_data)
            results = []
            print(f"Saved {idx} fighter profiles to {profile_csv}")
            time.sleep(10)  # Sleep for 10 seconds after every 100 saves
# with open(profile_csv, "w", newline="", encoding="utf-8") as csvfile:
with open(profile_csv, "a", newline="", encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    for fighter_data in results:
        writer.writerow(fighter_data)
print(f"All fighter profiles saved to {profile_csv}")

https://www.ufc.com/sitemap.xml?page=1 -> 26 fighter URLs
https://www.ufc.com/sitemap.xml?page=2 -> 74 fighter URLs
https://www.ufc.com/sitemap.xml?page=3 -> 43 fighter URLs
https://www.ufc.com/sitemap.xml?page=4 -> 48 fighter URLs
https://www.ufc.com/sitemap.xml?page=5 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=6 -> 56 fighter URLs
https://www.ufc.com/sitemap.xml?page=7 -> 12 fighter URLs
https://www.ufc.com/sitemap.xml?page=8 -> 190 fighter URLs
https://www.ufc.com/sitemap.xml?page=9 -> 60 fighter URLs
https://www.ufc.com/sitemap.xml?page=10 -> 33 fighter URLs
https://www.ufc.com/sitemap.xml?page=11 -> 17 fighter URLs
https://www.ufc.com/sitemap.xml?page=12 -> 510 fighter URLs
https://www.ufc.com/sitemap.xml?page=13 -> 67 fighter URLs
https://www.ufc.com/sitemap.xml?page=14 -> 116 fighter URLs
https://www.ufc.com/sitemap.xml?page=15 -> 774 fighter URLs
https://www.ufc.com/sitemap.xml?page=16 -> 1000 fighter URLs
https://www.ufc.com/sitemap.xml?page=17 -> 587 fighter URLs

---

# GitHub
##### [github.com/Greco1899/scrape_ufc_stats](https://github.com/Greco1899/scrape_ufc_stats)

In [12]:
# Download GitHub Data

os.makedirs('./data/github/', exist_ok=True)

urls = [
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_event_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_results.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fight_stats.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fighter_details.csv',
    'https://raw.githubusercontent.com/Greco1899/scrape_ufc_stats/main/ufc_fighter_tott.csv'
]

for url in urls:
    filename = os.path.join('./data/github/', url.split('/')[-1])
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {filename}")
    else:
        print(f"Failed to download: {url}")
        
# Print total rows in each file
for url in urls:
    filename = os.path.join('./data/github/', url.split('/')[-1])
    df = pd.read_csv(filename)
    print(f"Total rows in {filename}: {len(df)}")

Downloaded: ./data/github/ufc_event_details.csv
Downloaded: ./data/github/ufc_fight_details.csv
Downloaded: ./data/github/ufc_fight_results.csv
Downloaded: ./data/github/ufc_fight_stats.csv
Downloaded: ./data/github/ufc_fighter_details.csv
Downloaded: ./data/github/ufc_fighter_tott.csv
Total rows in ./data/github/ufc_event_details.csv: 721
Total rows in ./data/github/ufc_fight_details.csv: 8011
Total rows in ./data/github/ufc_fight_results.csv: 7998
Total rows in ./data/github/ufc_fight_stats.csv: 37602
Total rows in ./data/github/ufc_fighter_details.csv: 4272
Total rows in ./data/github/ufc_fighter_tott.csv: 4278


In [13]:
# GitHub: Cleaning

# Merge NICKNAME from ufc_fighter_details.csv to ufc_fighter_tott.csv
ufc_fighter_tott = pd.read_csv("data/github/ufc_fighter_tott.csv")
ufc_fighter_details = pd.read_csv("data/github/ufc_fighter_details.csv")
print(f"Total rows in ufc_fighter_tott: {len(ufc_fighter_tott)}")
print(f"Total rows in ufc_fighter_details: {len(ufc_fighter_details)}")
merged_df = pd.merge(ufc_fighter_tott, ufc_fighter_details[['URL', 'NICKNAME']], on='URL', how='left')
matching_count = merged_df['NICKNAME'].notnull().sum()
print(f"Found {matching_count} matching URL entries.")
print(f"Total rows in master: {len(merged_df)}")
merged_df.to_csv("data/github/master_fighters.csv", index=False)
os.remove("data/github/ufc_fighter_details.csv")
os.remove("data/github/ufc_fighter_tott.csv")
print("Deleted files: data/github/ufc_fighter_details.csv and data/github/ufc_fighter_tott.csv")

# Merge the DATE, LOCATION, and EVENT_URL of every fight from ufc_event_details.csv to ufc_fight_results.csv
os.remove("data/github/ufc_fight_details.csv")
print("Deleted, don't need this shit: data/github/ufc_fight_details.csv")
event_details_df = pd.read_csv('./data/github/ufc_event_details.csv')
event_details_df.rename(columns={'URL': 'EVENT_URL'}, inplace=True)
event_details_df.to_csv('./data/github/ufc_event_details.csv', index=False)
event_details_df = pd.read_csv('./data/github/ufc_event_details.csv')
fight_results_df = pd.read_csv('./data/github/ufc_fight_results.csv')
event_details_df['EVENT'] = event_details_df['EVENT'].str.strip()
fight_results_df['EVENT'] = fight_results_df['EVENT'].str.strip()
print(f"Rows in event_details_df: {len(event_details_df)}")
print(f"Rows in fight_results_df: {len(fight_results_df)}")
merged_df = pd.merge(fight_results_df, event_details_df[['EVENT', 'DATE', 'LOCATION', 'EVENT_URL']], on='EVENT', how='left')
merged_df.to_csv('./data/github/master_fights.csv', index=False)
matches_found = len(merged_df)
print(f"Rows in master_fights.csv: {matches_found}")
print('Saved to ./data/github/master_fights.csv')
os.remove("data/github/ufc_event_details.csv")
os.remove("data/github/ufc_fight_results.csv")
print("Deleted original files: data/github/ufc_event_details.csv and data/github/ufc_fight_results.csv")


Total rows in ufc_fighter_tott: 4278
Total rows in ufc_fighter_details: 4272
Found 2359 matching URL entries.
Total rows in master: 4278
Deleted files: data/github/ufc_fighter_details.csv and data/github/ufc_fighter_tott.csv
Deleted, don't need this shit: data/github/ufc_fight_details.csv
Rows in event_details_df: 721
Rows in fight_results_df: 7998
Rows in master_fights.csv: 7998
Saved to ./data/github/master_fights.csv
Deleted original files: data/github/ufc_event_details.csv and data/github/ufc_fight_results.csv


In [14]:
!open data/github/master_fights.csv
!open data/github/master_fighters.csv

In [15]:
# # Merge REACH and STANCE from ufc_fighter_tott.csv to fighter_info.csv

# fighter_info_df = pd.read_csv("data/fighter_info.csv", dtype=str)
# ufc_fighter_tott_df = pd.read_csv("data/github/ufc_fighter_tott.csv", dtype=str)

# # Standardize text formats (strip spaces and convert to lowercase)
# fighter_info_df["Fighter"] = fighter_info_df["Fighter"].str.strip().str.lower()
# fighter_info_df["Birth Date"] = fighter_info_df["Birth Date"].str.strip()

# ufc_fighter_tott_df["FIGHTER"] = ufc_fighter_tott_df["FIGHTER"].str.strip().str.lower()
# ufc_fighter_tott_df["DOB"] = ufc_fighter_tott_df["DOB"].str.strip()

# # Function to standardize date formats
# def standardize_date(date_str):
#     try:
#         return datetime.strptime(date_str, "%b %d, %Y").strftime("%Y-%m-%d")
#     except ValueError:
#         return None  # Return None for invalid/missing dates

# # Apply date standardization
# fighter_info_df["Birth Date"] = fighter_info_df["Birth Date"].apply(standardize_date)
# ufc_fighter_tott_df["DOB"] = ufc_fighter_tott_df["DOB"].apply(standardize_date)

# # Expand DOB matching with 1-day forward and backward
# def get_possible_dobs(dob):
#     try:
#         date_obj = datetime.strptime(dob, "%Y-%m-%d")
#         return [dob, (date_obj + timedelta(days=1)).strftime("%Y-%m-%d"), (date_obj - timedelta(days=1)).strftime("%Y-%m-%d")]
#     except (ValueError, TypeError):
#         return [dob]

# fighter_info_df["Possible DOBs"] = fighter_info_df["Birth Date"].apply(get_possible_dobs)

# # Explode dataframe to allow multiple DOB matches
# fighter_info_expanded = fighter_info_df.explode("Possible DOBs")

# # Select relevant columns from UFC fighter TOTT dataset
# ufc_fighter_tott_subset = ufc_fighter_tott_df[["FIGHTER", "REACH", "STANCE", "DOB"]]

# # Merge data on Fighter name and possible DOBs
# merged_fighter_info = fighter_info_expanded.merge(
#     ufc_fighter_tott_subset,
#     left_on=["Fighter", "Possible DOBs"],
#     right_on=["FIGHTER", "DOB"],
#     how="left"
# )

# # Drop duplicate columns and keep only the original rows from fighter_info.csv
# merged_fighter_info = merged_fighter_info.drop(columns=["FIGHTER", "DOB", "Possible DOBs"])

# # Ensure each fighter appears only once by keeping the first valid match
# merged_fighter_info = merged_fighter_info.drop_duplicates(subset=["Fighter"], keep="first")

# # Count the number of matched rows where REACH or STANCE were successfully merged
# matched_rows = merged_fighter_info.dropna(subset=["REACH", "STANCE"]).shape[0]

# # Save merged dataset
# merged_fighter_info.to_csv("data/fighter_info_with_reach_and_stance.csv", index=False)

# # Print the number of merged rows
# print(f"Merging complete. {matched_rows} rows found a match (including 1-day forward and backward check) and merged REACH and STANCE.")
# print("File saved as 'data/fighter_info_with_reach_and_stance.csv'.")



In [16]:
# GitHub: Cleaning

# fighter_info_df = pd.read_csv('./data/fighter_info.csv')
# event_data_df = pd.read_csv('./data/event_data_sherdog.csv')
# print(f"Number of rows in fighter_info.csv: {len(fighter_info_df)}")
# print(f"Number of rows in event_data_sherdog.csv: {len(event_data_df)}")

# # Remove leading/trailing whitespace, special characters, and normalize to lowercase
# fighter_info_df['Fighter'] = fighter_info_df['Fighter'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
# event_data_df['Fighter 1'] = event_data_df['Fighter 1'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)
# event_data_df['Fighter 2'] = event_data_df['Fighter 2'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True)

# # Handle missing values
# # fighter_info_df.fillna("Unknown", inplace=True)
# # event_data_df.fillna("Unknown", inplace=True)

# # Standardize column names
# # fighter_info_df.columns = fighter_info_df.columns.str.strip().str.lower().str.replace(' ', '_')
# # event_data_df.columns = event_data_df.columns.str.strip().str.lower().str.replace(' ', '_')

# # Remove duplicates
# print(f"Dropped {fighter_info_df.drop_duplicates(inplace=True)} duplicates from fighter_info_df")
# print(f"Dropped {event_data_df.drop_duplicates(inplace=True)} duplicates from event_data_df")

# # Save the cleaned datasets
# fighter_info_df.to_csv('./data/fighter_info.csv', index=False)
# event_data_df.to_csv('./data/event_data_sherdog.csv', index=False)
# print("Data cleaned and saved successfully.")
# print(f"Number of rows in fighter_info.csv: {len(fighter_info_df)}")
# print(f"Number of rows in event_data_sherdog.csv: {len(event_data_df)}")




---

# Tapology
##### [www.tapology.com](www.tapology.com)

In [17]:
import requests
import lxml.html
import pandas as pd
import os
from tqdm.notebook import tqdm
from xml.etree.ElementTree import ElementTree
from xml.etree.ElementTree import parse
from xml.etree.ElementTree import fromstring
import csv
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep
from random import uniform

### Create directories if they don't exist ###
os.makedirs('./data/tapology', exist_ok=True)

### Get sitemap.xml from tapology ###
sitemap_url = 'https://www.tapology.com/sitemap.xml'
response = requests.get(sitemap_url)
if response.status_code == 200:
    with open('./data/tapology/sitemap.xml', 'w') as file:
        file.write(response.text)
    print('Sitemap has been successfully downloaded and saved.')
else:
    print(f'Failed to download the sitemap. HTTP status code: {response.status_code}')
time.sleep(5)

### Get fighter URL's ###
urls = []
base_url = 'https://www.tapology.com/fightcenter/fighters/sitemap'
total_sitemaps = 114
for i in range(1, total_sitemaps + 1):
    sitemap_url = f'{base_url}_{i}.xml' if i > 1 else f'{base_url}.xml'
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        root = fromstring(response.content)
        for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
            loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc is not None:
                urls.append([loc.text])
        print(f'Processed sitemap {i}/{total_sitemaps}')
with open('./data/tapology/fighter_urls_tapology.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL'])
    writer.writerows(urls)
print('Finished saving fighter URLs to fighter_urls_tapology.csv')
time.sleep(5)

### Get bout URL's ###
urls = []
base_url = 'https://www.tapology.com/fightcenter/bouts/sitemap'
total_sitemaps = 222  # Adjusted for the total number of bout sitemaps mentioned
for i in range(1, total_sitemaps + 1):
    sitemap_url = f'{base_url}_{i}.xml' if i > 1 else f'{base_url}.xml'
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        root = fromstring(response.content)
        for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
            loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc is not None:
                urls.append([loc.text])
        print(f'Processed sitemap {i}/{total_sitemaps}')
        time.sleep(.5)
with open('./data/tapology/bout_urls_tapology.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL'])
    writer.writerows(urls)
print('Finished saving bout URLs to bout_urls_tapology.csv')
time.sleep(5)

### Get event URL's ###
urls = []
base_url = 'https://www.tapology.com/events/sitemap'
total_sitemaps = 35  # Total sitemaps from sitemap.xml to sitemap_35.xml as provided
for i in range(1, total_sitemaps + 1):
    sitemap_url = f'{base_url}.xml' if i == 1 else f'{base_url}_{i}.xml'
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        root = fromstring(response.content)
        for sitemap in root.findall('{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
            loc = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc is not None:
                urls.append([loc.text])
        print(f'Processed event sitemap {i}/{total_sitemaps}')
        time.sleep(.5)
with open('./data/tapology/event_urls_tapology.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['URL'])
    writer.writerows(urls)
print('Finished saving event URLs to event_urls_tapology.csv')


Sitemap has been successfully downloaded and saved.
Processed sitemap 1/114
Processed sitemap 2/114
Processed sitemap 3/114
Processed sitemap 4/114
Processed sitemap 5/114
Processed sitemap 6/114
Processed sitemap 7/114
Processed sitemap 8/114
Processed sitemap 9/114
Processed sitemap 10/114
Processed sitemap 11/114
Processed sitemap 12/114
Processed sitemap 13/114
Processed sitemap 14/114
Processed sitemap 15/114
Processed sitemap 16/114
Processed sitemap 17/114
Processed sitemap 18/114
Processed sitemap 19/114
Processed sitemap 20/114
Processed sitemap 21/114
Processed sitemap 22/114
Processed sitemap 23/114
Processed sitemap 24/114
Processed sitemap 25/114
Processed sitemap 26/114
Processed sitemap 27/114
Processed sitemap 28/114
Processed sitemap 29/114
Processed sitemap 30/114
Processed sitemap 31/114
Processed sitemap 32/114
Processed sitemap 33/114
Processed sitemap 34/114
Processed sitemap 35/114
Processed sitemap 36/114
Processed sitemap 37/114
Processed sitemap 38/114
Process

In [18]:
# Filter only ufc urls from event_urls_tapology.csv
# Scrape fighter names from all fights from those
df = pd.read_csv('data/tapology/event_urls_tapology.csv')
ufc_urls = df[df['URL'].str.contains('-ufc-', case=False, na=False) & ~df['URL'].str.contains('road-to-ufc', case=False, na=False) & ~df['URL'].str.contains('invitational', case=False, na=False)] 
ufc_urls.to_csv('data/tapology/event_urls_tapology_ufc.csv', index=False)
print(f"Filtered {len(ufc_urls)} UFC event URLs")

# # Only ufc urls from fighter_urls_tapology.csv
# df = pd.read_csv('data/tapology/bout_urls_tapology.csv')
# ufc_urls = df[df['URL'].str.contains('ufc', case=False, na=False)]
# ufc_urls.to_csv('data/tapology/bout_urls_tapology_ufc.csv', index=False)
# print(f"Filtered {len(ufc_urls)} UFC bout URLs")

Filtered 541 UFC event URLs


In [19]:
# Merge Tapology URL to fighter_info.csv and drop duplicates

# Load the datasets
fighter_info_path = "data/fighter_info.csv"
fighter_urls_path = "data/tapology/fighter_urls_tapology.csv"

df_fighter_info = pd.read_csv(fighter_info_path)
df_fighter_urls = pd.read_csv(fighter_urls_path)

# Extract fighter names from URLs
df_fighter_urls['Extracted_Name'] = df_fighter_urls['URL'].apply(
    lambda x: re.search(r'fighters/\d+-(.*)', x).group(1) if re.search(r'fighters/\d+-(.*)', x) else ''
)

# Format extracted names to be comparable (lowercase and replace dashes with spaces)
# df_fighter_urls['Formatted_Name'] = df_fighter_urls['Extracted_Name'].str.replace("-", " ").str.lower()
df_fighter_urls['Formatted_Name'] = df_fighter_urls['Extracted_Name'].str.replace("-", " ")

# Format fighter names from fighter_info.csv for comparison
# df_fighter_info['Formatted_Fighter'] = df_fighter_info['Fighter'].str.lower()
df_fighter_info['Formatted_Fighter'] = df_fighter_info['Fighter']

# Merge datasets on formatted fighter names, only including rows where a match is found
df_merged = df_fighter_info.merge(
    df_fighter_urls[['Formatted_Name', 'URL']],
    left_on='Formatted_Fighter',
    right_on='Formatted_Name',
    how='inner'
)

# Drop duplicates based on all columns
df_merged.drop_duplicates(inplace=True)

df_merged.drop(columns=['Formatted_Fighter', 'Formatted_Name'], inplace=True)
updated_file_path = "data/tapology/fighter_info_with_urls.csv"
df_merged.to_csv(updated_file_path, index=False)
print(f"Updated file saved: {updated_file_path}")

# Print the number of matches found
matches_found = len(df_merged)
print(f"Number of matches found: {matches_found}")


Updated file saved: data/tapology/fighter_info_with_urls.csv
Number of matches found: 14


In [20]:
!open data/github/*
!open data/event_data_sherdog.csv
!open data/fighter_info.csv